In [2]:
!pip install spacy


  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/3d/0b/5d26c2156f2f8ed4e45f3de65cab6f4a4707bf5c7ab5fd6cb4e0388696e3/spacy-3.8.3-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached spacy-3.8.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (27 kB)
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/e5/1f/5

In [4]:
!python -m spacy download en_core_web_sm

/Users/archismankandu/Desktop/Rick/Binary/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
pip install pytesseract opencv-python-headless pillow numpy

  Obtaining dependency information for pytesseract from https://files.pythonhosted.org/packages/7a/33/8312d7ce74670c9d39a532b2c246a853861120486be9443eebf048043637/pytesseract-0.3.13-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/dc/53/2c50afa0b1e05ecdb4603818e85f7d174e683d874ef63a6abe3ac92220c8/opencv_python_headless-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata
  Obtaining dependency information for pillow from https://files.pythonhosted.org/packages/a6/62/c7b359e924dca274173b04922ac06aa63614f7e934d132f2fe1d852509aa/pillow-11.1.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached pillow-11.1.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 5.0 MB/s eta 0:00:0000:0100:01
Using cached pillow-11.1.0-cp39-cp39-macosx_11_0_arm64.whl (3.1 MB)

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip insta

In [11]:
pip install opencv-python-headless


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path, handwritten=False):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if handwritten:
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        kernel = np.ones((3, 3), np.uint8)
        processed_image = cv2.dilate(binary, kernel, iterations=1)
        processed_image = cv2.erode(processed_image, kernel, iterations=1)
    else:
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
        kernel = np.ones((1, 1), np.uint8)
        processed_image = cv2.dilate(binary, kernel, iterations=1)
        processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path, handwritten=False):
    preprocessed_image = preprocess_image(image_path, handwritten)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        if handwritten:
            config = '--psm 6'
            text = pytesseract.image_to_string(pil_image, config=config)
        else:
            text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    for keyword in keywords:
        if keyword in ["Name:", "Customer Name", "Recipient", "Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Date:", "Date"]:
            pattern = re.compile(rf"{keyword}\s*([\d/]+)")
        elif keyword in ["Amount:", "Amount"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+(?:.\d+)?)")
        elif keyword in ["ID:", "ID"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+)")
        else:
            continue

        for match in pattern.finditer(text):
            extracted_text = match.group(1).strip()
            extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def save_to_database(data, document_id):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        for item in data:
            query = "INSERT INTO extracted_data (document_id, text, label) VALUES (%s, %s, %s)"
            values = (document_id, item["Text"], item["Label"])
            cursor.execute(query, values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path, handwritten=False):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path, handwritten)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Address:", "Date:", "Amount:", "ID:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                cnx = mysql.connector.connect(**DB_CONFIG)
                cursor = cnx.cursor()
                insert_doc_query = "INSERT INTO documents (file_path) VALUES (%s)"
                doc_values = (file_path,)
                cursor.execute(insert_doc_query, doc_values)
                cnx.commit()
                document_id = cursor.lastrowid
                cursor.close()
                cnx.close()

                save_to_database(important_info, document_id)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()
handwritten_checkbox = widgets.Checkbox(value=False, description='Handwritten')

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected, handwritten=handwritten_checkbox.value)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, handwritten_checkbox, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

/Users/archismankandu/Desktop/Rick/Binary/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


FileChooser(path='/Users/archismankandu/Desktop/Rick/Binary', filename='', title='', show_hidden=False, select…

Checkbox(value=False, description='Handwritten')

Output()